In [34]:
%run Functions.ipynb
import statsmodels.formula.api as smf 
plt.style.use(["seaborn-dark-palette", "seaborn-whitegrid"])

### Hedge Fund Benchmarks: A Risk-Based Approach regression

#### Trend-Following Risk Factors (3):
- Bond Trend-Following Factor
- Currency Trend-Following Factor
- Commodity Trend-Following Factor

https://faculty.fuqua.duke.edu/~dah7/HFRFData.htm

Do not find daily data.

####  Equity Market Factor 
Standard & Poors 500 stock return. https://finance.yahoo.com/quote/%5EGSPC/history?period1=1437350400&period2=1595203200&interval=1mo&filter=history&frequency=1d

In [35]:
def Ret(df,col): return (  df[col] / df[col].shift(1) -1 )

gspc = pd.read_csv("^GSPC_daily.csv")
gspc["Equity_market_factor"] = Ret(gspc, "Close")

#### The Size Spread Factor
Russell 2000 index monthly total return - Standard & Poors 500 monthly total return. https://finance.yahoo.com/quote/%5ERUT/history?period1=1437350400&period2=1595203200&interval=1d&filter=history&frequency=1d

In [36]:
rut = pd.read_csv("^RUT_daily.csv")
rut["ret"] = Ret(rut, "Close")

rut["Size_spread_factor"] = rut["ret"] - gspc["Equity_market_factor"]

#### The Bond Market Factor - DGS10 (calculate change)
https://fred.stlouisfed.org/series/DGS10 

In [37]:
bond_market_factor = pd.read_csv("DGS10.csv")
bond_market_factor["Date"] = bond_market_factor["DATE"]
bond_market_factor = bond_market_factor[bond_market_factor["DGS10"] != "."]
bond_market_factor["DGS10"] = bond_market_factor["DGS10"].astype(float)
bond_market_factor["Bond_market_factor"] = Ret(bond_market_factor, "DGS10")

#### The Credit Spread Factor DGS10/DBA
 The monthly change in the Moody's Baa yield less 10-year treasury constant maturity yield (month end-to-month end)
https://fred.stlouisfed.org/series/DBAA

In [38]:
credit_spread_factor = pd.read_csv("DBAA.csv")
credit_spread_factor["Date"] = credit_spread_factor["DATE"]
credit_spread_factor = credit_spread_factor[credit_spread_factor["DBAA"] != "."]
credit_spread_factor["DBAA"] = credit_spread_factor["DBAA"].astype(float)
credit_spread_factor["change"] = Ret(credit_spread_factor, "DBAA")
credit_spread_factor["Credit_spread_factor"] = credit_spread_factor["change"] - bond_market_factor["Bond_market_factor"]

#### The Emerging Market Index EEM
MSCI Emerging Market index monthly total return https://it.finance.yahoo.com/quote/EEM/history?period1=1437350400&period2=1595203200&interval=1d&filter=history&frequency=1d

In [39]:
eem = pd.read_csv("EEM_daily.csv")
eem["Emergency_market_factor"] = Ret(eem, "Close")

Creating the factors dataset

In [40]:
factors8 = pd.merge(gspc[["Date","Equity_market_factor"]], rut[["Date","Size_spread_factor"]], how="left", on="Date")
factors8 = pd.merge(factors8, bond_market_factor[["Date","Bond_market_factor"]], how="left", on="Date")
factors8 = pd.merge(factors8, credit_spread_factor[["Date","Credit_spread_factor"]], how="left", on="Date")
factors8 = pd.merge(factors8, eem[["Date","Emergency_market_factor"]], how="left", on="Date")

factors8 = factors8.set_index("Date")
factors8 = factors8[factors8.index>"2015-08-01"]
factors8 = factors8*100

In [41]:
my_returns = pd.read_csv("my_returns_daily.csv") # insert my returns
my_returns["Date"] = my_returns["date"]
my_returns["return_from_trade"] = my_returns["return_from_trade"]*100

In [42]:
factors8 = pd.merge(my_returns[["Date","return_from_trade"]], factors8, how="left", on="Date").fillna(0)

In [43]:
ff = smf.ols(formula='return_from_trade ~ Equity_market_factor + Size_spread_factor + Bond_market_factor + Credit_spread_factor + Emergency_market_factor', data=factors8)
res = ff.fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      return_from_trade   R-squared:                       0.032
Model:                            OLS   Adj. R-squared:                  0.026
Method:                 Least Squares   F-statistic:                     4.882
Date:                Wed, 16 Sep 2020   Prob (F-statistic):           0.000211
Time:                        19:28:20   Log-Likelihood:                -2491.3
No. Observations:                 737   AIC:                             4995.
Df Residuals:                     731   BIC:                             5022.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
===========================================================================================
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                   0.8050      0.264      3.052      0.002       0.287       1.323
Equity_market_factor       -0.5317      0.383     -1.388      0.166      -1.284       0.220
Size_spread_factor         -0.0392      0.409     -0.096      0.924      -0.842       0.763
Bond_market_factor          1.2009      0.280      4.291      0.000       0.651       1.750
Credit_spread_factor        1.3097      0.344      3.808      0.000       0.634       1.985
Emergency_market_factor     0.2021      0.356      0.568      0.570      -0.496       0.901
==============================================================================
Omnibus:                      190.404   Durbin-Watson:                   1.855
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1175.331
Skew:                           1.008   Prob(JB):                    6.03e-256
Kurtosis:                       8.849   Cond. No.                         9.75
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""